In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sqlite3 as sql3

In [2]:
# Get data from URL

url      = "https://en.wikipedia.org/wiki/List_of_Grand_Slam_men%27s_singles_champions#Champions_list"
response = requests.get(url,headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"})

# Extract content from the response

hyperText = BeautifulSoup(response.content,'html.parser')

# Find relevant anchor points in the HTML
dataTable = hyperText.find('table',class_="wikitable sortable mw-datatable sticky-header sort-under-center")

# Get table headers
headers = []
for hi in dataTable.find_all('th')[0:9]:
    headers.append(hi.get_text().strip().replace('\xad', '')) # Remove potential soft hyphenation

# Replace 'AE' and 'OE' headers for readability

headers[2] = 'Amateur Era'
headers[3] = 'Open Era'

# Loop through all the data

rowData = []

for idx in range(0,40):
        rowInfo = []
        ri = dataTable.find_all('tr')[1:][idx].find_all('th')
        
        # The table has an attribute called "rowspan" that repeats the number of titles if a player
        # has the same number of Grand Slams won. This is not stored in the tag <th> as player with
        # a unique number of won Grand Slam; it returns an empty list so the next piece of code 
        # stores the repeated value and appends it into the list
        
        if ri: # For pieces of the HTML where th is not empty
            if 'rowspan' in ri[0].attrs: # where the attribute 'rowspan' exists
                repGS   = ri[0].get_text().replace('\n',"") # Get the missing value in the next players
                rii     = dataTable.find_all('tr')[1:][idx].find_all(['th','td'])[0:10]
                rowInfo = [riii.get_text().replace('\n',"") for riii in rii]
            else: # For players with unique values
                rii     = dataTable.find_all('tr')[1:][idx].find_all(['th','td'])[0:10]
                rowInfo = [riii.get_text().replace('\n',"") for riii in rii]
            if rowInfo:
                 rowData.append(rowInfo)
        else: # For players that share the same number of Grand Slams
            rii     = dataTable.find_all('tr')[1:][idx].find_all('td')[0:10]
            rowInfo = [riii.get_text().replace('\n',"") for riii in rii]
            rowInfo.insert(0,repGS)
            if rowInfo:
                rowData.append(rowInfo)

In [6]:
# Generate data frame

dfGS = pd.DataFrame(rowData,columns=headers)

# Inserting a ranking column
dfGS.insert(loc=0,column='Rank',value=(np.linspace(1,40,40,endpoint=True,dtype=int)))
dfGS['Titles'] = dfGS['Titles'].astype('int')
dfGS['Australian Open'] = dfGS['Australian Open'].astype('int')
dfGS['French Open'] = dfGS['French Open'].astype('int')
dfGS['Wimbledon'] = dfGS['Wimbledon'].astype('int')
dfGS['US Open'] = dfGS['US Open'].astype('int')
dfGS['Amateur Era'] = dfGS['Amateur Era'].astype('int',errors='ignore')

# Convert data frame to SQL

connection = sql3.connect("web_db.db")

# Load data to SQL

dfGS.to_sql("tennis",connection)
cursor = connection.cursor()